In [1]:
import numpy as np
from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
import tensorflow as tf
from multiprocessing.dummy import Pool as ThreadPool 

In [2]:
class ModelClient():
    
    MODEL_HOST = '10.100.241.46'
    PORT=8500
    
    def __init__(self, model_host = MODEL_HOST, port=PORT):
        channel = implementations.insecure_channel(model_host, int(port))
        stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
        self.stub = stub
        
    def predict(self, indata):
        request = predict_pb2.PredictRequest()
        request.model_spec.name = 'model'
        request.model_spec.signature_name = 'serving_default'
        request.inputs['input'].CopyFrom(tf.contrib.util.make_tensor_proto(indata, shape=indata.shape))
        response = self.stub.Predict.future(request, 10.0)  # wait max 5s
        return response.result().outputs["score"].float_val


In [3]:
nb_predictions = 1000000
batch_size = 400
nb_batches = int(nb_predictions / batch_size)

modelClient = ModelClient()
indata = np.random.randint(100000, size=(batch_size, 20)).astype(np.float32)

indata_batches = []
for i in range(nb_batches):
    indata_batches.append(indata)
    
print('Total number of predictions:', nb_predictions, 'Batchsize:', batch_size, 'Number of batches:', nb_batches)

Total number of predictions: 1000000 Batchsize: 400 Number of batches: 2500


/usr/local/lib/python3.5/dist-packages/tensorflow_serving/apis/prediction_service_pb2.py:131: DeprecationWarning: beta_create_PredictionService_stub() method is deprecated. This method will be removed in near future versions of TF Serving. Please switch to GA gRPC API in prediction_service_pb2_grpc.
  'prediction_service_pb2_grpc.', DeprecationWarning)


In [4]:
def make_prediction(indata):
    #print("Request", indata.shape)
    result = modelClient.predict(indata)
    #print("Response", result.shape)

In [6]:
pool = ThreadPool(36) 

%time result = pool.map(make_prediction, indata_batches)

CPU times: user 1.37 s, sys: 298 ms, total: 1.67 s
Wall time: 14.2 s


In [18]:
1000000 / 14.5

68965.5172413793